# Functions

## What are functions?

Functions are computational devices that transform input _values_ into output _values_, and do nothing _else_.

In [1]:
// `add one` function

def addOne(x: Int): Int = 
    x+1

defined function addOne

If we run this function, the only thing that happens is the computation of a new value:

In [2]:
addOne(4)

res1: Int = 5

Functions that do something else, besides returning values, are called _impure_ functions. Functional programming deals only with _pure_, or mathematical, functions.

In [3]:
// An impure function

def impureAdd(input: Int): Int = {
    println("computing add one")
    return input + 1
}


defined function impureAdd

If we run this function, we will see an _effect_ in the console (besides the pure computation of `input + 1`): 

In [4]:
impureAdd(5)

computing add one


res3: Int = 6

There are many kinds of effects: writing to the console, reading from the keyworkd, reading from a socket, calling a web service, executing a query over the database, etc. Clearly, we need effects if we want our programs to do something useful, so pure functions alone are not enough. We will talk about this later on.



## Functions as modularity devices

Why are functions so important in programming? Because they help us to _modularize_ our code. For instance, let's consider the following programs, which access the following data structure of key-value pairs (we will talk about this structure in detail later on):

In [5]:
val config: Map[String, String] = 
    Map("URL" -> "http://hablapps.com",
        "PORT" -> "8080")

config: Map[String, String] = Map(
  "URL" -> "http://hablapps.com",
  "PORT" -> "8080"
)

Our first program access the configuration data for the value of the "URL" key. If it's not found, then the default value "default.url" is returned (similarly, we will discuss the `match` keyword further in the course).

In [6]:
// Program 1
val url: String = config.get("URL") match {
  case Some(u) => u
  case None => "default.url"
}

url: String = "http://hablapps.com"

Our second program accesses the configuration data for the value of the "PORT" key. If it's not found, then the default value "8080" is returned.

In [7]:
// Program 2
val port: String = config.get("PORT") match {
  case Some(p) => p
  case None => "8080"
}

port: String = "8080"

These two programs do _almost_ the same. The only differences lie in the particular keys and default values the programs refer to, but, otherwise, they do the same thing. However, this _common factor_ is not reflected in the code. Indeed, we may get one program from the other by copy-pasting, a clear signal of [code-smell](https://en.wikipedia.org/wiki/Code_smell).

These programs are _monolythic_, in the sense that they are not made by composing large enough modules. In this case, the common logic of the program and the values it operates on are intermingled in the same code. 

How can we abstract away the differences and package the common logic in a single module? With functions:

In [11]:
def getValue(key: String, default: String): String = 
    config.get(key) match {
      case Some(p) => p
      case None => default
    }

defined function getValue

In [12]:
getValue

cmd12.sc:1: missing argument list for method getValue in class Helper
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `getValue _` or `getValue(_,_)` instead of `getValue`.
val res12 = getValue
            ^Compilation Failed

: 

This is an abstract module which we can combine with other modules to get back the very same functionality:

In [9]:
// Program 1
val url: String = getValue("URL", "default.url")

url: String = "http://hablapps.com"

In this case, we combine the module `getKeyFrom` with the modules (data values and variables, in particular) `config`, `"URL"` and `"default.url"`. The composition method is just simple function application.

Which are the advantages of using functions? As in the general case, having a more modular solution enables _reuse_, particularly of those modules which are abstract or parameterised. For instance, we can benefit from this level of reuse by re-implementing the `url` program in the following way:

In [10]:
// Program 2
val port: String = getValue("PORT", "8080")

port: String = "8080"

## Functions as methods

In an object-oriented language, functions are implemented through _methods_, i.e. using the `def` keyword. Note that these methods are invariably part of an `object`, `class` or `trait` declaration. Typically, pure functions are declared as part of objects. For instance, we may declare a set of arithmetic functions as follows: 

In [13]:
import scala.math.{pow, Pi}

object Areas{
    
    def circle(radius: Double): Double = 
        Pi*pow(radius,2)
    
    def rectangle(width: Double, height: Double): Double = 
        width + height
}

import scala.math.{pow, Pi}


defined object Areas

In notebooks and the Scala REPL, `def` declarations appear to be independent from any object or class, but they are not:

In [ ]:
def foo(i: Int): Int = i
// show errors: "missing argument list for method foo in class Helper"


When we study higher-order functions, we will see that functions in Scala can also be represented as _objects_, i.e. not only as methods. However, that representation also builds essentially upon methods.

## Functions as values

Functions can also be represented as _values_, i.e. as objects. This allows us to implement functions that receive other functions as arguments, or return functions as results. This special functions are called _higher-order functions_ (HOF), and they feature as a great modularity device. We will mainly discuss this feature of HOFs in PF-3; now, we just want to focus on how are functions actually represented as values in a OO language like Scala. 

This representation builds essentially upon methods, in particular, _reified_ methods. For instance, let's consider the following functions:

In [17]:
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 

defined function addOneM
defined function substractOneM

We want to implement a HOF that receives an integer-to-integer function, such as `addOneM`and `substractOneM`, and calls this function over a given number. We may want to write something like this:

In [13]:
def call(def int2int(n: Int): Int, number: Int): Int =
   int2int(number)

(console):1:10 expected ")"
def call(def int2int(n: Int): Int, number: Int): Int =
         ^

: 

where the first argument `int2int` attempts to represent any function that receives an integer and returns another integer. 

This code is not legal in Scala, but we can create a new class whose only method is the function that we want to pass around:

In [32]:
// Reify and abstract the implementation!
trait FunctionIntToInt{
    def apply(number: Int): Int
}

val addOneV: FunctionIntToInt = new FunctionIntToInt{
    def apply(number: Int): Int = 
        number + 1 
}

val substractOneV: FunctionIntToInt = new FunctionIntToInt{
    def apply(number: Int): Int = 
        number - 1 
}



defined trait FunctionIntToInt
addOneV: FunctionIntToInt = ammonite.$sess.cmd31$Helper$$anon$1@65fb33c2
substractOneV: FunctionIntToInt = ammonite.$sess.cmd31$Helper$$anon$2@3264abc8

In [33]:
1: Int
addOneV : FunctionIntToInt

addOneV(5)

// addOneM: Any
//addOneV.apply(5)
//addOneV(5)
//addOneM(5)

res32_0: Int = 1
res32_1: FunctionIntToInt = ammonite.$sess.cmd31$Helper$$anon$1@65fb33c2
res32_2: Int = 6

Now, we can implement the `call` HOF as follows: 

In [34]:
def call(int2int: FunctionIntToInt, number: Int): Int =
   int2int(number)

defined function call

In order to use this HOF with the `addOneM` and `substractOneM` functions, we must create reified versions for them: 

In [34]:
/*
def addOneM(number: Int): Int = 
    number + 1

def substractOneM(number: Int): Int = 
    number - 1 
*/

We call the `addOneV` and `substractOneV` function-values, i.e. functions represented as values. Now, we can use the `call` HOF as follows:

In [35]:
call(addOneV, 6)

res34: Int = 7

In [36]:
call(substractOneV, 5)

res35: Int = 4

In [39]:
call(addOneM, 6)

res38: Int = 7

In [38]:
call(new FunctionIntToInt{ def apply(i: Int): Int = addOneM(i) }, 6)

res37: Int = 7

## Standard functions in Scala

The Scala programming language offers many facilities to work with functions as values. First, the standard library provides the following _generic_ types [`Function1`](https://www.scala-lang.org/api/current/scala/Function1.html), [`Function2`](https://www.scala-lang.org/api/current/scala/Function2.html), etc., roughly implemented as follows:

In [ ]:
object Std{
    /*
    abstract class FunctionInt2Int{
        def apply(number: Int): Int
    }
    */

    // up to Function22
}


Using these standard classes, we can create the `addOneV` function-value in a similar way than before: 

In [ ]:
/*
val addOneV: FunctionInt2Int = new FunctionInt2Int{
    def apply(number: Int): Int = 
        number + 1
}
*/

But we can do it more easily, since Scala also provides special syntax to declare function types and instantiate functions (so-called _lambda expressions_):

In [ ]:
/*
val addOneV: Function1[Int, Int] = new Function1[Int, Int]{
    def apply(a: Int): Int = 
        a + 1
}
*/

// val substractOneV

And we can also profit from type inference:

Using these syntactic facilities, the `call` HOF has a more appealing signature: 

In [ ]:
/*
def call(int2int: FunctionInt2Int, number: Int): Int = 
    int2int.apply(number)
*/

which we can use as follows:

And we can even pass function-methods that are converted on the fly to function-values!

This is the so-called _eta-expansion_.

## Currying

We can use a similar syntax for functions of two arguments. So, the function-value equivalent of this function-method: 

In [ ]:
def sumM(x: Int, y: Int): Int = 
    x+y

could be written in this way: 

In [ ]:
// val sumV

but also in the following one:

In [ ]:
// val sumV with sugar

or, exploiting type inference:

However, `Function2`, `Function3`, ... classes are not extrictly necessary, and sometimes we can get along with `Function1`. But, how can we create a function of two arguments with `Function1` alone? The trick is the following:

In [ ]:
/*
val sumC: (Int, Int) => Int = 
    (a: Int, b: Int) => a + b
*/

Note that brackets in `Int => (Int => Int)` are used for clarity, but are not needed. Basically, we created a function of one argument that returns another function of one argument. So, the expression: 

returns a function that can be applied again:

We can apply this strategy to functions of any number of arguments. This is called _currying_ and _currified functions_. The analog in function-methods is [multiple-parameter lists](https://docs.scala-lang.org/tour/multiple-parameter-lists.html):

In [ ]:
/*
def sum(x: Int, y: Int): Int = 
    x+y
*/

## Functions compose

We can create new functions by composing other functions whose signatures match. This is great from a modularity perspective. For instance, the following function is implemented in a non-modular way:

In [ ]:
def isEvenLength: String => Boolean = 
    ???

This function is somehow the combination of two more basic functions `length` and `isEven`:

but this is not reflected in the current implementation. How can we redefine the function `isEvenLength` using the functions `length` and `isEven`? We can use a HOF which helps us to compose functions:

Then, we can redefine `isEvenLength` in a modular way from the `length` and `isEven` building blocks:

In [ ]:
// val isEvenLength: String => Boolean = ???

The HOF `compose` is actually defined by `Function1`: 

In [ ]:
// val isEvenLength: String => Boolean = ???

or using infix notation:

In [ ]:
// val isEvenLength: String => Boolean = ???

Note that a similar function to `compose`, called `andThen`, is also available in the standard library: 

In [ ]:
// val isEvenLength: String => Boolean = ???

Also, there is a function which behaves as the identity element with respect to the operation `compose`, i.e. no matter which other function we choose to compose with the `identity` function, the result will be that function:
1. `identity[B] compose f == f` for all `f: A => B`
2. `f compose identity[A] == f` for all `f: A => B`

## Contravariance and variance in Function traits

Trait `Function1` is actually defined as follows:

```trait Function1[-A, +B]{
   def apply(a: A): B
}```

In [ ]:
abstract class Animal
class Perro extends Animal
class Doberman extends Perro
class Gato extends Animal
class Siames extends Gato

In [ ]:
def fGP: Gato => Perro = ??? 
def fGD: Gato => Doberman = ??? 
def fGA: Gato => Animal = ???
def fAP: Animal => Perro = ???
def fSP: Siames => Perro = ??? 
def fAD: Animal => Doberman = ???

Which functions can be passed to this method?

In [ ]:
def f2(f: Gato => Perro) = ???

In [ ]:
// def r1 = f2(fGP)
// def r2 = f2(fGD)
// def r3 = f2(fGA)
// def r4 = f2(fAP)
// def r5 = f2(fSP)
// def r6 = f2(fAD)